In [2]:
import os
import json
import pandas as pd
import traceback

In [5]:
from langchain.chat_models import ChatOpenAI

In [ ]:
KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=0.3)